# 26.基于MindSpore实现AC算法

本实验主要介绍使用MindSpore实现(AC)Actor-Critic算法，并使用自定义数据进行测试。

# 1.实验目的
- 理解AC算法原理。
- 理解Actor和Critic部分的具体工作流程。
- 自定义数据进行训练，根据奖励结果判断学习效果。

# 2. 算法知识点介绍
Actor-Critic 算法合并了以策略为基础的 Policy Gradient和以值为基础的 Q-Learning 两类强化学习算法，该算法中将前者当作 Actor，用来基于概率选择行为。将后者当作 Critic，用来评判 Actor 的行为得分，然后 Actor 又会根据 Critic 的评分修改行为的概率。这使得它既可以在有效的处理连续动作的选取，又可以进行单步更新（PG算法的回合更新降低了学习效率）。

下面分别介绍一下 Actor 网络和 Critic 网络这个两个部分。

## 2.1 Actor网络
Actor网络采用的是基于策略的Policy-Grandient算法。PG算法的损失函数可以表示为 $loss = -E[\log [\pi (a|s)]\cdot \psi]$, 其中 $\psi$ 是对某个轨迹的评分，在AC算法中它可以有多种表示方式：  
1. 状态价值函数：$V(s)$  
2. 动作价值函数：$Q(s,a)$
3. TD-error: $r+\gamma \cdot Q(s_{t+1}, a_{t+1}) - Q(s_t, a_t)$
4. 优势函数：$V(s,a)$
5. 总回报：$\sum r_t$
6. 加入基线的总回报：$\sum r_t -b$  
  
根据梯度策略算法的定义：策略优化目标函数如以下公式所示：
$$
L_{\pi} = \sum_{a \in A}{\log \pi_{\theta}(s_t, a_t)(G_{t}^{n} - V(s_t))}
$$
令 $advantage = G_{t}^{n} - V(s_t)$, 称 advantage 为优势函数。采用n步时序差分法求解时，$G_t$ 可以用公式表示如下：
$$
G_t = R_{t+1} + \gamma R_{t+2} + \cdots + \gamma^{n-1} R_{t+n} + \gamma^n V(s_{t+n})
$$
通过观察可以看出，当n为一个完整的状态序列大小时，该算法与蒙特卡洛算法等价。  
    
在实际编写代码时，采用TD-error作为评价，同时为了鼓励探索(exploration), 在损失函数中还加入了交叉熵损失。  


## 2.2 Critic网络
Critic网络采用的是基于值函数的Q-Learning算法，采用 $loss = (TD_{error})^2$ 作为Critic网络的损失函数。  
整个Actor-Critic算法流程如下图所示：  
  
![AC算法流程图](Figures/fig_001.png)  


Actor-Critic算法流程：  
准备：评价网络学习率 $\alpha_1$ 和策略网络的学习率 $\alpha_2$;
1. 随机初始化评价网络参数 $\theta_q$ 和策略网络参数 $\theta_{\pi}$;
2. 重复以下操作至最大次数：
3. 智能体与环境交互n步并收集由状态、动作、奖励构成的序列$\{s_0, a_0, r_1, s_1, a_1, r_2,\cdots \}$;  
4. 计算Critic网络的损失值 $L_i^{critic} = L_{critic}(s, r)$;  
5. 计算Actor网络的损失值 $L_{actor}^i = L_{actor}(s, a, advantage)$;  
6. 更新Critic网络参数 $\theta_q \leftarrow \theta_q - \alpha_1 \nabla_{\theta}\sum_{i}L^{i}_{critic}$;    
7. 更新Actor网络参数 $\theta_{\pi} \leftarrow  \theta_{\pi} - \alpha_2 \nabla_{\pi}\sum_{i}L^{i}_{Actor}$;   
8. 结束。


## 2.3 算法评价
- 优点：可以进行单步更新，不需要跑完一个episode再更新网络参数，相较于传统的PG（Policy Gradient）更新更快。传统PG方法对于价值的估计虽然是无偏估计，但方差较大，AC方法允许少量偏差，但是能够有效降低方差。
- 缺点：Actor的行为取决于Critic的Value，但是因为Critic本身就很难收敛，与Actor结合后收敛难度会更大。



## 2.4 gym平台介绍
本实验借助了gym平台的环境，该平台由 openai 公司开发，且提供了一整套与平台中虚拟环境进行交互的 api接口，gym 的推出为强化学习算法的研究提供了更好地基准测试平台，同时将各类 环境标准化，使研究员可以专注于算法研究而无需花过多的时间在环境的模拟上。gym 提供一个 step 函数供智能体与环境进行交互，其参数为动作，主要返回值及含义分别为：  
- observation：表示智能体所处环境的当前状态，代表着智能体的观察值，例如棋盘上棋子的状态。
- reward：表示智能体采取操作后从环境中获得的奖励，其类型可能是整数、小数等，但是具体的规模和类型与具体的规模和类型与环境有关，但是智能体的总目标仍然是获取最大的奖励值。
- done: 大多数任务都属于阶段性任务，当到达一定条件的时候表示任务已经结束，比如五子棋游戏中的一方五子相连，机器人在路面上摔倒，或者在规定的步数以内没有完成任务，则都代表任务结束。所以 done 是一个判断条件，类型为布尔值，代表当前任务是否结束，如果结束则可以选择使用 reset 函数重置当前任务。

# 3. 实验环境
在动手进行实践之前，需要注意以下几点：
* 确保实验环境正确安装，包括安装MindSpore。安装过程：首先登录[MindSpore官网安装页面](https://www.mindspore.cn/install)，根据安装指南下载安装包及查询相关文档。同时，官网环境安装也可以按下表说明找到对应环境搭建文档链接，根据环境搭建手册配置对应的实验环境。
* 推荐使用交互式的计算环境Jupyter Notebook，其交互性强，易于可视化，适合频繁修改的数据分析实验环境。
* 实验也可以在华为云一站式的AI开发平台ModelArts上完成。
* 推荐实验环境：MindSpore版本=2.4；Python环境=3.11


|  硬件平台 |  操作系统  |          软件环境           | 开发环境 | 环境搭建链接 |
| :-----:| :----: |:-----------------------:|:----:   |:----:   |
| CPU | Windows-x64 | MindSpore2.4 Python3.11 | JupyterNotebook |[MindSpore环境搭建实验手册第二章2.1节和第三章3.1节](./MindSpore环境搭建实验手册.docx)|
| Ascend 910  | Linux-x86_64| MindSpore2.4 Python3.11 | JupyterNotebook |[MindSpore环境搭建实验手册第四章](./MindSpore环境搭建实验手册.docx)|

# 4 数据处理
本实验采用Open Gym中的CartPole-v1环境，实际参考了论文：[Neuronlike Adaptive Elements That Can Solve Difficult Learning Control Problem](https://ieeexplore.ieee.org/document/6313077).中的倒立摆控制问题。  


## 4.1 数据准备  
  
CartPole即车杆游戏，游戏模型如下图所示。游戏里面有一个小车，上面竖着一根杆子，每次重置后的初始状态会有所不同。小车需要左右移动来保持杆子竖直，为了保证游戏继续运行需要满足以下两个条件：    
  
<img src="./Figures/fig_002.png" width="40%">    
  
- 杆子的倾斜角 $\theta$ 不能大于15°;  
- 小车的移动范围 $x$ 需要保持在一定范围（中间到两边各2.4个长度单位）;
  
动作（action）：
- 左移（0）;
- 右移（1）;  
  
状态变量（state variables）:
- x: 小车在轨道上的位置（position of the cart on the track）;
- $\theta$: 杆子与竖直方向的夹角（angle of the pole with the vertical）;
- $\dot{x}$: 小车速度（cart velocity）;
- $\dot{\theta}$: 角度变化率（rate of change of the angle）;

## 4.2 数据加载

通过导入gym库加载CartPole-v1游戏场景，在gym的CartPole环境中，小车执行左移或右移的action后，env会返回一个+1的reward。本实验使用的CartPole-v1在达到500个reward后，游戏会结束。加载车杆游戏代码如下所示：

In [12]:
# gym强化学习库, gym版本为0.21.0
import gym
# 加载车杆游戏场景
env=gym.make('CartPole-v1')
# 获取状态数
state_number=env.observation_space.shape[0]
# 获取动作数
action_number=env.action_space.n

# 5.算法实现
使用MindSpore实现AC算法并进行训练。
## 5.1 导入Python库并配置运行信息

In [13]:
# os库
import os
# 引入MindSpore
import mindspore
# 引入神经网络模块
import mindspore.nn as nn
# 常见算子操作
import mindspore.ops as F
# 引入numpy
import numpy as np
# 引入张量模块
from mindspore import Tensor
# 配置静态库
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

## 5.2 定义参数和超参数
定义训练回合数、Actor学习率、Critic学习率以及折现因子 $\gamma$，代码如下：

In [14]:
round_num = 10     # 训练回合数 
LR_A = 0.005        # learning rate for actor
LR_C = 0.01         # learning rate for critic
Gamma = 0.9         # 折现因子

## 5.3 定义损失函数
继承自nn.LossBase类构造损失函数，代码如下：

In [15]:
# 定义损失函数MAELoss
class MAELoss(nn.LossBase):
    def __init__(self, reduction="none"):
        super(MAELoss, self).__init__(reduction)
    # 构造损失函数
    def construct(self, prob, a,td):
        a_constant=a
        log_prob = F.log(prob)
        td_constant=td[0]
        # 定义信息熵
        log_prob_constant=-log_prob[0][a_constant]
        # 计算演员损失函数
        actor_loss = -log_prob_constant * td_constant
        return actor_loss
mae = MAELoss()

## 5.4 构建Actor网络

构建Actor网络类以及Actor类，构建Actor的行为选择函数和学习函数，代码如下：

In [16]:
# Actor网络类
class ActorNet(nn.Cell):
    def __init__(self):
        super(ActorNet, self).__init__()
        # 全连接层1
        self.fc1 = nn.Dense(state_number, 50)
        # 全连接层2
        self.fc2 = nn.Dense(50, 20)
        # 全连接层3
        self.fc3 = nn.Dense(20, action_number)
        # ReLU激活函数
        self.relu = nn.ReLU()
        # Sigmoid激活函数
        self.sigmoid = nn.Sigmoid()
        # Softmax归一化函数
        self.softmax = nn.Softmax()
    # 构造Actor网络
    def construct(self, x):
        # 全连接层
        x = self.fc1(x)
        # ReLU激活函数
        x = self.relu(x)
        # 全连接层
        x = self.fc2(x)
        # Sigmoid激活函数
        x = self.sigmoid(x)
        # 全连接层
        x = self.fc3(x)
        # 返回softmax函数结果
        return self.softmax(x)
actor_net = ActorNet()
actor_net.update_parameters_name("actor")
# 演员类
# class Actor():
#     def __init__(self):
#         # 构造演员网络
#         self.actor=ActorNet()
#         # 优化器
#         self.optimizer = nn.Adam(self.actor.trainable_params(),learning_rate=LR_A)
#         # 损失函数
#         self.loss=MAELoss()
#     # 行为选择函数
#     def choose(self,inputstate):
#         # inputstate=Tensor(inputstate)
#         # 输入状态
#         inputstate=Tensor([inputstate])
#         probs=self.actor(inputstate).asnumpy()
#         # 获取行为
#         action=np.random.choice(np.arange(action_number),p=probs[0])
#         # 返回行为
#         return action
#     # 学习函数
#     def learn(self,s,a,td):
#         s = Tensor([s])
#         prob = self.actor(s)
#         # log_prob = F.log(prob)
#         # td张量化
#         td=Tensor(td)
#         # a转为整型变量
#         a=int(a)
#         a_constant = a
#         # 信息熵
#         log_prob = F.log(prob)
#         td_constant = td[0]
#         log_prob_constant = -log_prob[0][a_constant]
#         # 构造损失网络
#         loss_net=nn.WithLossCell(self.actor,loss_fn=self.loss(prob,Tensor(a),td))
#         # 构造训练网络
#         train_net = nn.TrainOneStepCell(loss_net,self.optimizer)

定义行为选择函数

In [17]:
def choose(inputstate):
        # inputstate=Tensor(inputstate)
        # 输入状态
    inputstate=Tensor([inputstate])
    probs=actor_net(inputstate).asnumpy()
        # 获取行为
    action=np.random.choice(np.arange(action_number),p=probs[0])
        # 返回行为
    return action

## 5.5 构建Critic网络
构建Critic网络类以及Critic类，构建Critic的学习函数。

In [18]:
# 评论者网络
class CriticNet(nn.Cell):
    def __init__(self):
        super(CriticNet, self).__init__()
        # 全连接层1
        self.fc1 = nn.Dense(state_number, 50)
        # 全连接层2
        self.fc2 = nn.Dense(50, 20)
        # 全连接层3
        self.fc3 = nn.Dense(20, 1)
        # ReLU激活函数
        self.relu = nn.ReLU()
        # Sigmoid激活函数
        self.sigmoid = nn.Sigmoid()
        # Softmax归一化函数
        self.softmax = nn.Softmax()
    # 构造函数
    def construct(self, x):
        # 全连接层
        x = self.fc1(x)
        # ReLU激活函数
        x = self.relu(x)
        # 全连接层
        x = self.fc2(x)
        # Sigmoid激活函数
        x = self.sigmoid(x)
        # 返回全连接层结果
        return self.fc3(x)
critic_net = CriticNet()
critic_net.update_parameters_name("critic")
crtic_loss = nn.MSELoss()
# class Critic():
#     def __init__(self):
#         self.critic=CriticNet()
#         self.tempA=ActorNet()
#         # 优化器
#         self.optimizer = nn.Adam(self.critic.trainable_params(),learning_rate=LR_C)
#         # 均方误差（MSE）
#         self.lossfunc=nn.MSELoss()
#     def learn(self,s,r,s_):
#         s = Tensor([s])
#         # 输入当前状态，由网络得到估计v
#         v=self.critic(s)
#         r=Tensor([r])
#         s_ = Tensor([s_])
#         temp=Tensor(self.critic(s_).asnumpy())
#         # 真实v
#         reality_v=r+Gamma*temp[0]
#         # 构造损失网络
#         loss_net=nn.WithLossCell(self.critic,self.lossfunc(reality_v,v[0]))
#         # 构造训练网络
#         train_net=nn.TrainOneStepCell(loss_net,self.optimizer)
#         # 计算真实v与估计v之间的差距
#         advantage=(reality_v-v).asnumpy()
#         return advantage

损失函数和优化器

In [19]:
optimizer_c = nn.Adam(critic_net.trainable_params(),learning_rate=LR_C)
optimizer_c.update_parameters_name("critic")
optimizer_a = nn.Adam(actor_net.trainable_params(),learning_rate=LR_A)
optimizer_a.update_parameters_name("actor")

# 6 模型训练
首先实例化演员类和评论者类，然后开始训练。每轮训练迭代过程中需要重置环境，打印出每个行为所得到的奖励。

In [20]:
def actor_foward(s, a, td):
    s = Tensor([s])
    prob = actor_net(s)

    td=Tensor(td)
    a=int(a)
    a_constant = a
        # 信息熵
    log_prob = F.log(prob)
    return mae(prob,Tensor(a),td)

def critic_foward(s, r, s_):
     s = Tensor([s])
     # 输入当前状态，由网络得到估计v
     v=critic_net(s)
     r=Tensor([r])
     s_ = Tensor([s_])
     temp=Tensor(critic_net(s_).asnumpy())
     # 真实v
     reality_v=r+Gamma*temp[0]
    # 构造损失网络
     loss=crtic_loss(reality_v,v[0])

     advantage=(reality_v-v).asnumpy()
     return loss, advantage

grad_a = mindspore.value_and_grad(actor_foward, None, actor_net.trainable_params())
grad_c = mindspore.value_and_grad(critic_foward, None, critic_net.trainable_params(), has_aux=True)

def train_step(observation,reward,observation_,action):
    (loss_c,td_error), grads_c = grad_c(observation,reward,observation_)
    optimizer_c(grads_c)
    loss_a,grads_a = grad_a(observation,action,td_error)
    optimizer_a(grads_a)
    return loss_a, loss_c



In [21]:
print('AC训练中...')
# 开始训练
for i in range(round_num):
    r_totle=[]
    # 环境重置
    observation = env.reset()#环境重置
    # 判断数据类型（gym版本）
    observation = observation if type(observation)== np.ndarray else observation[0]
    # print(observation)
    # print(observation)
    while True:
        # 选择行为
        action=choose(observation)
        # print(env.step(action))
        step = env.step(action)
        # 获取训练过程相关参数
        observation_ = step[0]
        reward = step[1]
        done = step[2]
        info = step[3]

        #单步训练
        train_step(observation,reward,observation_,action)

        observation=observation_
        # 行为选择奖励加入总回报
        r_totle.append(reward)
        # done = True进行下一轮训练
        if done:
            break
    # 计算总回报
    r_sum=sum(r_totle)
    # 打印训练回合数和奖励
    print("\r回合数：{} 奖励：{}".format(i,r_sum),end=" ")
    print(observation)
    # 保存检查点
    if i%100==0 and i > int(round_num/2):
        mindspore.save_checkpoint(actor_net, "./actor.ckpt")
        mindspore.save_checkpoint(critic_net, "./critic.ckpt")
print('AC训练完成')


AC训练中...
回合数：0 奖励：15.0 [-0.19966501 -1.3352216   0.24848944  2.191414  ]
回合数：1 奖励：13.0 [-0.17569318 -1.3318977   0.22890553  2.2851973 ]
回合数：2 奖励：25.0 [-0.19067861 -0.1848725   0.21366093  0.79066277]
回合数：3 奖励：13.0 [-0.22458123 -1.7879466   0.24441074  2.7920365 ]
回合数：4 奖励：12.0 [-0.13601221 -1.593982    0.24318342  2.6496403 ]
回合数：5 奖励：15.0 [-0.1508342  -1.3270633   0.21148017  2.236424  ]
回合数：6 奖励：10.0 [-0.12669736 -1.5193003   0.22161996  2.4622717 ]
回合数：7 奖励：9.0 [-0.09981839 -1.3250958   0.21616352  2.2072396 ]
回合数：8 奖励：16.0 [-0.1865989 -1.1938583  0.2273462  1.9863337]
回合数：9 奖励：20.0 [-0.19446789 -1.6037664   0.21610154  2.2189066 ]
AC训练完成
